## CAT BOOST REGRESSOR

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import zipfile
import re
from scipy import stats
from catboost import CatBoostRegressor
from tqdm import tqdm
import gc
import datetime as dt

<font color = blue> Once we have imported the necessary libraries, we are going to define our data path and then, load the data: </font>

In [2]:
data_path = './data/'

In [3]:
print('Loading Properties ...')
properties2016 = pd.read_csv(data_path + 'properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv(data_path + 'properties_2017.csv', low_memory = False)

Loading Properties ...


In [4]:
properties2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [5]:
properties2016.shape

(2985217, 58)

In [6]:
properties2017.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [7]:
properties2017.shape

(2985217, 58)

<font color = blue> We are going to specify the parameter 'parse_dates' to make the column 'transactiondate' in train files date formatted: </font>

In [8]:
print('Loading Train ...')
train2016 = pd.read_csv(data_path + 'train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv(data_path + 'train_2017.csv', parse_dates=['transactiondate'], low_memory=False)

Loading Train ...


In [9]:
train2016.shape

(90275, 3)

In [10]:
train2017.shape

(77613, 3)

In [11]:
train2016.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


<font color = blue> Now we are going to parse dates to get one column for year, one for month, one for day and one for quarter: </font>

In [12]:
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = (df["transactiondate"].dt.year - 2016)*4 + df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

In [13]:
train2016 = add_date_features(train2016)
train2017 = add_date_features(train2017)

In [14]:
train2016.head()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter
0,11016594,0.0276,2016,1,1,1
1,14366692,-0.1684,2016,1,1,1
2,12098116,-0.0040,2016,1,1,1
3,12643413,0.0218,2016,1,2,1
4,14432541,-0.0050,2016,1,2,1


In [15]:
print('Loading Sample ...')
sample_submission = pd.read_csv(data_path + 'sample_submission.csv', low_memory = False)

Loading Sample ...


<font color = blue> Our sample submission will be a file with the parcel id for each property and six prediction columns for different dates (last 2016 quarter and last 2017 quarter): </font>

In [16]:
sample_submission.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


<font color = blue> As we are going to apply the first model, cat boost, to the column for 201712 predictions, we remove the other columns: </font>

In [17]:
sample_submission_model_6 = sample_submission.drop(columns = ['201610', '201611', '201612', '201710', '201711'])
sample_submission_model_6.head()

,ParcelId,201712
0,10754147,0
1,10759547,0
2,10843547,0
3,10859147,0
4,10879947,0


<font color = blue> We will merge properties and train files to get all data grouped: </font>

In [18]:
print('Merging Train with Properties ...')
train2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')
train2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

Merging Train with Properties ...


In [19]:
train2016.head()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016,1,1,1,1.0,NaN,NaN,2.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016,1,1,1,NaN,NaN,NaN,3.5,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016,1,1,1,1.0,NaN,NaN,3.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016,1,2,1,1.0,NaN,NaN,2.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016,1,2,1,NaN,NaN,NaN,2.5,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [20]:
train2016.shape

(90275, 63)

In [21]:
train2017.head()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017,13,1,5,NaN,NaN,NaN,3.5,...,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017,13,1,5,NaN,NaN,NaN,1.0,...,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13
2,14186244,0.005383,2017,13,1,5,NaN,NaN,NaN,2.0,...,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13
3,12177905,-0.103410,2017,13,1,5,NaN,NaN,NaN,3.0,...,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13
4,10887214,0.006940,2017,13,1,5,1.0,NaN,NaN,3.0,...,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13


In [22]:
train2017.shape

(77613, 63)

<font color = blue> Now we will generate our training and test files to run the model, the training file will be all properties and train from 2016 and 2017, while for the test, we will merge the sample submission with the unique properties file: </font>

In [23]:
print('Concat Train 2016 & 2017 ...')
train_df = pd.concat([train2016, train2017], axis = 0)
test_df = pd.merge(sample_submission_model_6[['ParcelId']], properties2016.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')


Concat Train 2016 & 2017 ...


In [24]:
train_df.head()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016,1,1,1,1.0,NaN,NaN,2.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016,1,1,1,NaN,NaN,NaN,3.5,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016,1,1,1,1.0,NaN,NaN,3.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016,1,2,1,1.0,NaN,NaN,2.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016,1,2,1,NaN,NaN,NaN,2.5,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [25]:
train_df.shape

(167888, 63)

In [26]:
test_df.head()

,ParcelId,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [27]:
test_df.shape

(2985217, 58)

<font color = blue> To optimize memory management, we are using garbage collections algorithms (GC), which solve reference cycles (when one or more objects are referencing each other) that reference counting cannot detect: </font>

In [28]:
del properties2016, properties2017, train2016, train2017
gc.collect();

<font color = blue> Let's do some feature engineering. We have to deal with missing values, where we will establish a threshold of 98% missing values to remove those fields, and also with features with one unique value and others that we do not want to use in our training: </font>

In [29]:
print('Missing data fields to remove ...')
missing_perc_thresh = 0.98
exclude_missing = []
num_rows = train_df.shape[0]
for c in train_df.columns:
    num_missing = train_df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print(exclude_missing)
print("We exclude: %s" % len(exclude_missing))

Missing data fields to remove ...
['architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid', 'decktypeid', 'finishedsquarefeet13', 'finishedsquarefeet6', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'storytypeid', 'typeconstructiontypeid', 'yardbuildingsqft26', 'fireplaceflag']
We exclude: 13


In [30]:
print ("Remove features with one unique value ...")
exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)
print(exclude_unique)
print("We exclude: %s" % len(exclude_unique))

Remove features with one unique value ...
['decktypeid', 'hashottuborspa', 'poolcnt', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'storytypeid', 'fireplaceflag', 'taxdelinquencyflag']
We exclude: 9


In [31]:
del num_rows, missing_perc_thresh
gc.collect();

In [32]:
print ("Define training features ...")
exclude_other = ['parcelid', 'logerror', 'propertyzoningdesc']
train_features = []
for c in train_df.columns:
    if c not in exclude_missing \
       and c not in exclude_other:
        train_features.append(c)
print(train_features)
print("We use these for training: %s" % len(train_features))

Define training features ...
['transaction_year', 'transaction_month', 'transaction_day', 'transaction_quarter', 'airconditioningtypeid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet15', 'finishedsquarefeet50', 'fips', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'pooltypeid7', 'propertycountylandusecode', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr', 'unitcnt', 'yardbuildingsqft17', 'yearbuilt', 'numberofstories', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount', 'taxdelinquencyflag', 'taxdelinquencyyear', 'censustractandblock']
We use these for training: 47


<font color = blue> Now we have to deal with categorical features: </font>

In [33]:
print ("Define categorical features ...")
cat_feature_inds = []
cat_unique_thresh = 100
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

Define categorical features ...
Cat features are: ['transaction_year', 'transaction_month', 'transaction_day', 'transaction_quarter', 'airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'hashottuborspa', 'heatingorsystemtypeid', 'pooltypeid7', 'propertycountylandusecode', 'propertylandusetypeid', 'regionidcounty', 'assessmentyear', 'taxdelinquencyflag', 'taxdelinquencyyear']


<font color = blue> As we are going to use a tree based model, we will replace NaN values by '-999' so they do not interfere with proper data: </font>

In [34]:
print ("Replacing NaN values by -999 ...")
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Replacing NaN values by -999 ...


In [35]:
print ("Training model 6: Cat boost ...")
X_train = train_df[train_features]
y_train = train_df.logerror
print(X_train.shape, y_train.shape)

test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = add_date_features(test_df)
X_test = test_df[train_features]
print(X_test.shape)

Training model 6: Cat boost ...
(167888, 47) (167888,)
(2985217, 47)


In [36]:
y_pred = 0.0
model = CatBoostRegressor(
    iterations = 690, learning_rate = 0.01,
    depth = 7, l2_leaf_reg = 5,
    loss_function = 'MAE',
    eval_metric = 'MAE',
    random_seed = i)
model.fit(
    X_train, y_train,
    cat_features = cat_feature_inds)
y_pred += model.predict(X_test)


0:	learn: 0.0691066	total: 854ms	remaining: 9m 48s
1:	learn: 0.0690008	total: 1.67s	remaining: 9m 35s
2:	learn: 0.0689181	total: 2.36s	remaining: 9m
3:	learn: 0.0688466	total: 3.25s	remaining: 9m 17s
4:	learn: 0.0687940	total: 4.15s	remaining: 9m 28s
5:	learn: 0.0687412	total: 5.18s	remaining: 9m 51s
6:	learn: 0.0686985	total: 6.49s	remaining: 10m 33s
7:	learn: 0.0686657	total: 7.18s	remaining: 10m 12s
8:	learn: 0.0686317	total: 7.78s	remaining: 9m 48s
9:	learn: 0.0686027	total: 8.55s	remaining: 9m 41s
10:	learn: 0.0685720	total: 9.22s	remaining: 9m 29s
11:	learn: 0.0685450	total: 9.9s	remaining: 9m 19s
12:	learn: 0.0685255	total: 10.7s	remaining: 9m 16s
13:	learn: 0.0685004	total: 11.3s	remaining: 9m 5s
14:	learn: 0.0684852	total: 12s	remaining: 8m 58s
15:	learn: 0.0684710	total: 12.7s	remaining: 8m 55s
16:	learn: 0.0684529	total: 13.6s	remaining: 8m 57s
17:	learn: 0.0684336	total: 14.4s	remaining: 8m 57s
18:	learn: 0.0684252	total: 15.5s	remaining: 9m 7s
19:	learn: 0.0684146	total: 1

157:	learn: 0.0678011	total: 2m 19s	remaining: 7m 50s
158:	learn: 0.0677996	total: 2m 20s	remaining: 7m 48s
159:	learn: 0.0677970	total: 2m 20s	remaining: 7m 46s
160:	learn: 0.0677945	total: 2m 21s	remaining: 7m 44s
161:	learn: 0.0677935	total: 2m 22s	remaining: 7m 43s
162:	learn: 0.0677906	total: 2m 22s	remaining: 7m 42s
163:	learn: 0.0677862	total: 2m 23s	remaining: 7m 40s
164:	learn: 0.0677844	total: 2m 24s	remaining: 7m 41s
165:	learn: 0.0677825	total: 2m 25s	remaining: 7m 39s
166:	learn: 0.0677803	total: 2m 26s	remaining: 7m 38s
167:	learn: 0.0677794	total: 2m 26s	remaining: 7m 36s
168:	learn: 0.0677769	total: 2m 27s	remaining: 7m 34s
169:	learn: 0.0677754	total: 2m 28s	remaining: 7m 33s
170:	learn: 0.0677723	total: 2m 29s	remaining: 7m 32s
171:	learn: 0.0677688	total: 2m 29s	remaining: 7m 30s
172:	learn: 0.0677675	total: 2m 30s	remaining: 7m 29s
173:	learn: 0.0677656	total: 2m 31s	remaining: 7m 27s
174:	learn: 0.0677605	total: 2m 31s	remaining: 7m 26s
175:	learn: 0.0677585	total:

310:	learn: 0.0675276	total: 4m 19s	remaining: 5m 15s
311:	learn: 0.0675270	total: 4m 19s	remaining: 5m 14s
312:	learn: 0.0675266	total: 4m 20s	remaining: 5m 13s
313:	learn: 0.0675246	total: 4m 21s	remaining: 5m 12s
314:	learn: 0.0675241	total: 4m 21s	remaining: 5m 11s
315:	learn: 0.0675219	total: 4m 23s	remaining: 5m 11s
316:	learn: 0.0675210	total: 4m 24s	remaining: 5m 10s
317:	learn: 0.0675189	total: 4m 24s	remaining: 5m 9s
318:	learn: 0.0675183	total: 4m 25s	remaining: 5m 8s
319:	learn: 0.0675173	total: 4m 26s	remaining: 5m 7s
320:	learn: 0.0675163	total: 4m 26s	remaining: 5m 6s
321:	learn: 0.0675148	total: 4m 27s	remaining: 5m 5s
322:	learn: 0.0675138	total: 4m 28s	remaining: 5m 4s
323:	learn: 0.0675119	total: 4m 28s	remaining: 5m 3s
324:	learn: 0.0675098	total: 4m 29s	remaining: 5m 2s
325:	learn: 0.0675076	total: 4m 30s	remaining: 5m 1s
326:	learn: 0.0675067	total: 4m 30s	remaining: 5m
327:	learn: 0.0675052	total: 4m 31s	remaining: 5m
328:	learn: 0.0675021	total: 4m 33s	remaining

463:	learn: 0.0673176	total: 5m 38s	remaining: 2m 44s
464:	learn: 0.0673172	total: 5m 39s	remaining: 2m 44s
465:	learn: 0.0673163	total: 5m 39s	remaining: 2m 43s
466:	learn: 0.0673143	total: 5m 39s	remaining: 2m 42s
467:	learn: 0.0673117	total: 5m 40s	remaining: 2m 41s
468:	learn: 0.0673112	total: 5m 40s	remaining: 2m 40s
469:	learn: 0.0673077	total: 5m 41s	remaining: 2m 39s
470:	learn: 0.0673061	total: 5m 41s	remaining: 2m 38s
471:	learn: 0.0673043	total: 5m 42s	remaining: 2m 38s
472:	learn: 0.0673038	total: 5m 42s	remaining: 2m 37s
473:	learn: 0.0673031	total: 5m 43s	remaining: 2m 36s
474:	learn: 0.0673021	total: 5m 43s	remaining: 2m 35s
475:	learn: 0.0673006	total: 5m 43s	remaining: 2m 34s
476:	learn: 0.0672972	total: 5m 44s	remaining: 2m 33s
477:	learn: 0.0672951	total: 5m 45s	remaining: 2m 33s
478:	learn: 0.0672932	total: 5m 45s	remaining: 2m 32s
479:	learn: 0.0672929	total: 5m 46s	remaining: 2m 31s
480:	learn: 0.0672920	total: 5m 46s	remaining: 2m 30s
481:	learn: 0.0672915	total:

617:	learn: 0.0671298	total: 7m 4s	remaining: 49.4s
618:	learn: 0.0671290	total: 7m 4s	remaining: 48.7s
619:	learn: 0.0671279	total: 7m 5s	remaining: 48s
620:	learn: 0.0671270	total: 7m 5s	remaining: 47.3s
621:	learn: 0.0671268	total: 7m 5s	remaining: 46.5s
622:	learn: 0.0671255	total: 7m 6s	remaining: 45.8s
623:	learn: 0.0671253	total: 7m 6s	remaining: 45.1s
624:	learn: 0.0671230	total: 7m 6s	remaining: 44.4s
625:	learn: 0.0671221	total: 7m 7s	remaining: 43.7s
626:	learn: 0.0671216	total: 7m 7s	remaining: 43s
627:	learn: 0.0671190	total: 7m 8s	remaining: 42.3s
628:	learn: 0.0671176	total: 7m 8s	remaining: 41.6s
629:	learn: 0.0671171	total: 7m 9s	remaining: 40.9s
630:	learn: 0.0671160	total: 7m 9s	remaining: 40.2s
631:	learn: 0.0671137	total: 7m 9s	remaining: 39.5s
632:	learn: 0.0671115	total: 7m 11s	remaining: 38.8s
633:	learn: 0.0671115	total: 7m 11s	remaining: 38.1s
634:	learn: 0.0671111	total: 7m 11s	remaining: 37.4s
635:	learn: 0.0671101	total: 7m 12s	remaining: 36.7s
636:	learn: 

In [41]:
submission = pd.DataFrame({
    'ParcelId': test_df['ParcelId'],
})
test_dates = {
    '201712': pd.Timestamp('2017-11-30')
}
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    submission[label] = y_pred
    
submission.to_csv(data_path + 'Model_6_CatBoost.csv', float_format='%.6f',index=False)

Predicting for: 201712 ... 


In [42]:
predictions_model_6 = pd.read_csv(data_path + 'Model_6_CatBoost.csv')
predictions_model_6.head()

,ParcelId,201712
0,10754147,0.017177
1,10759547,0.001478
2,10843547,0.026582
3,10859147,0.034545
4,10879947,0.005915
